In [ ]:
#!import "./Extensions"

In [ ]:
using System.Linq.Expressions;

# Exchange Rates

In [ ]:
public static async Task<Dictionary<string, Dictionary<FxPeriod, double>>> GetExchangeRatesDictionaryAsync(this IQuerySource querySource, int year, int month)
    => (await querySource.Query<ExchangeRate>()
    .Where(x => x.Year == year - 1 && x.Month == MonthInAYear && x.FxType == FxType.Spot ||
                x.Year == year && x.Month == month)
    .ToArrayAsync())
    .ToDictionaryGrouped(x => x.Currency,
                         x => x.ToDictionary(y => (y.Year, y.Month, y.FxType) switch
                                             {
                                                 (_, _, _) when y.Year == year - 1 && y.Month == MonthInAYear && y.FxType == FxType.Spot    => FxPeriod.BeginningOfPeriod,
                                                 (_, _, _) when y.Year == year     && y.Month == month        && y.FxType == FxType.Average => FxPeriod.Average,
                                                 (_, _, _) when y.Year == year     && y.Month == month        && y.FxType == FxType.Spot    => FxPeriod.EndOfPeriod
                                             },
                                             y => y.FxToGroupCurrency));

# Current and Previous Parameters

In [ ]:
public static async Task<T[]> LoadParameterAsync<T>(
    this IQuerySource querySource,
    int year,
    int month,
    Expression<Func<T, bool>> filterExpression = null ) 
    where T : IWithYearAndMonth
{
    return await querySource.Query<T>()
                     .Where(x => x.Year == year && x.Month <= month || x.Year < year)
                     .Where(filterExpression?? (Expression<Func<T, bool>>)(x => true))
                     .ToArrayAsync();
}

In [ ]:
public static async Task<Dictionary<string, T>> LoadCurrentParameterAsync<T> (
    this IQuerySource querySource,
    Args args,
    Func<T, string> identityExpression,
    Expression<Func<T, bool>> filterExpression = null ) 
    where T : IWithYearMonthAndScenario
{
    return (await querySource.LoadParameterAsync<T>(args.Year, args.Month, filterExpression))
                    .Where(yc => yc.Scenario == args.Scenario || yc.Scenario == null)
                    .GroupBy(yc => identityExpression)
                    .Select(yc => yc.OrderByDescending(x => x.Year)
                                    .ThenByDescending(x => x.Month)
                                    .ThenByDescending(x => x.Scenario)
                                    .FirstOrDefault())
                                    .ToDictionary(identityExpression);
}

In [ ]:
public static async Task<Dictionary<string, Dictionary<int, T>>> LoadCurrentAndPreviousParameterAsync<T> (
    this IQuerySource querySource,
    Args args,
    Func<T, string> identityExpression,
    Expression<Func<T, bool>> filterExpression = null ) 
    where T : IWithYearMonthAndScenario
{
    var parameters = (await querySource.LoadParameterAsync<T>(args.Year, args.Month, filterExpression))
                    .Where(yc => yc.Scenario == args.Scenario || yc.Scenario == null)
                    .GroupBy(identityExpression);
                                         
    var ret = new Dictionary<string, Dictionary<int, T>>();
    foreach(var p in parameters)
    {
        var inner = ret.GetOrAdd(p.Key, _ => new Dictionary<int, T>());

        var currentCandidate = p.Where(x => x.Year == args.Year).OrderByDescending(x => x.Month).ThenByDescending(x => x.Scenario).FirstOrDefault();
        var previousCandidate = p.Where(x => x.Year < args.Year).Where(x => x.Scenario == null).OrderByDescending(x => x.Year).ThenByDescending(x => x.Month).FirstOrDefault();
    
        inner.Add(CurrentPeriod, currentCandidate != null ? currentCandidate : previousCandidate);
        inner.Add(PreviousPeriod, previousCandidate != null ? previousCandidate : currentCandidate);
    }
    return ret;
}

<a id='yield-curve'></a>
# Yield Curve

## Locked-in

In [ ]:
public static async Task<Dictionary<string, YieldCurve>> LoadLockedInYieldCurveAsync(this IQuerySource querySource, Args args, 
                                                                                                       IEnumerable<DataNodeData> dataNodes)
{
    var lockedInYieldCurveByGoc = new Dictionary<string, YieldCurve>();
    foreach (var dn in dataNodes.Where(x => x.ValuationApproach == ValuationApproaches.BBA))
    {
        var argsNew = args with {Year = dn.Year, Month = dn.Month, Scenario = dn.Scenario};
        var loadedYc = (await querySource.LoadCurrentParameterAsync<YieldCurve>(argsNew, (x => x.Currency), x => x.Currency == dn.ContractualCurrency));
        
        if(!loadedYc.TryGetValue(dn.ContractualCurrency, out var lockedYc))
            ApplicationMessage.Log(Error.YieldCurveNotFound, dn.ContractualCurrency, argsNew.Year.ToString(), argsNew.Month.ToString());
            
        lockedInYieldCurveByGoc[dn.DataNode] = lockedYc;
    }
    
    return lockedInYieldCurveByGoc;
}

## Current

In [ ]:
public static async Task<Dictionary<string, Dictionary<int, YieldCurve>>> LoadCurrentYieldCurveAsync(this IQuerySource querySource, Args args, 
                                                                                                      IEnumerable<DataNodeData> dataNodes)
{
    var contractualCurrenciesInScope = dataNodes.Select(dn => dn.ContractualCurrency).ToHashSet();
    return (await querySource.LoadCurrentAndPreviousParameterAsync<YieldCurve>(args, 
                                                                               x => x.Currency,
                                                                               x => contractualCurrenciesInScope.Contains(x.Currency)
                                                                               ));
}

# Data Node State

In [ ]:
public static async Task<Dictionary<string, DataNodeState>> LoadDataNodeStateAsync(this IQuerySource querySource, Args args)
{
    return (await querySource.LoadCurrentAndPreviousParameterAsync<DataNodeState>(args, x => x.DataNode))
                             .Where(x => x.Value[CurrentPeriod].State != State.Inactive)
                             .ToDictionary(x => x.Key, x => x.Value[CurrentPeriod]);
}

# Data Nodes

In [ ]:
public static async Task<Dictionary<string, DataNodeData>> LoadDataNodesAsync(this IQuerySource querySource, Args args)
{
    var dataNodeStates = await querySource.LoadCurrentAndPreviousParameterAsync<DataNodeState>(args, x => x.DataNode);
    var activeDataNodes = dataNodeStates.Where(kvp => kvp.Value[CurrentPeriod].State != State.Inactive).Select(kvp => kvp.Key);
    
    return (await querySource.Query<GroupOfContract>().Where(dn => activeDataNodes.Contains(dn.SystemName)).ToArrayAsync())
                            .ToDictionary(dn => dn.SystemName, dn => {
                                                                      var dnCurrentState = dataNodeStates[dn.SystemName][CurrentPeriod];
                                                                      var dnPreviousState = dataNodeStates[dn.SystemName][PreviousPeriod];
                                                                      return new DataNodeData(){Year = dnPreviousState.Year, 
                                                                                                  Month = dnPreviousState.Month,
                                                                                                  State = dnCurrentState.State,
                                                                                                  PreviousState = dnPreviousState.State,
                                                                                                  DataNode = dn.SystemName,
                                                                                                  ContractualCurrency  = dn.ContractualCurrency,
                                                                                                  FunctionalCurrency  = dn.FunctionalCurrency,
                                                                                                  LineOfBusiness  = dn.LineOfBusiness,
                                                                                                  ValuationApproach  = dn.ValuationApproach,
                                                                                                  OciType  = dn.OciType,
                                                                                                  Portfolio  = dn.Portfolio,
                                                                                                  AnnualCohort  = dn.AnnualCohort,
                                                                                                  LiabilityType  = dn.LiabilityType,
                                                                                                  Profitability  = dn.Profitability,
                                                                                                  Partner  = dn.Partner,
                                                                                                  IsReinsurance  = dn.GetType() == typeof(GroupOfReinsuranceContract),
                                                                                                  };
                                                                     }
                                         );
}

# Data Node Parameters

## Single data Node

In [ ]:
public static async Task<Dictionary<string, Dictionary<int, SingleDataNodeParameter>>> LoadSingleDataNodeParametersAsync(this IQuerySource querySource, Args args)
{
    return await querySource.LoadCurrentAndPreviousParameterAsync<SingleDataNodeParameter>(args, x => x.DataNode);
}

## Inter data Node

In [ ]:
public static async Task<Dictionary<string, Dictionary<int, HashSet<InterDataNodeParameter>>>> LoadInterDataNodeParametersAsync(this IQuerySource querySource, Args args)
{
    var identityExpressions = new Func<InterDataNodeParameter, string>[]{x => x.DataNode, x => x.LinkedDataNode,};
    var parameterArray = (await querySource.LoadParameterAsync<InterDataNodeParameter>(args.Year, args.Month));
    var parameters = identityExpressions.SelectMany(ie => parameterArray.GroupBy(ie));
                                         
    return parameters.SelectMany(p => p
                                 .GroupBy(x => x.DataNode != p.Key ? x.DataNode : x.LinkedDataNode)
                                 .Select(gg =>
                                         {
                                             var currentCandidate = gg.Where(x => x.Year == args.Year).OrderByDescending(x => x.Month).FirstOrDefault();
                                             var previousCandidate = gg.Where(x => x.Year < args.Year).OrderByDescending(x => x.Year).ThenByDescending(x => x.Month).FirstOrDefault();
                                             return (key: p.Key,
                                                     currentPeriod: currentCandidate != null ? currentCandidate : previousCandidate,
                                                     previousPeriod: previousCandidate != null ? previousCandidate : currentCandidate);
                                         })
                                )
        .ToDictionaryGrouped(x => x.key,
                             x => new Dictionary<int, HashSet<InterDataNodeParameter>>{ {CurrentPeriod, x.Select(y => y.currentPeriod).ToHashSet()},
                                                                                       {PreviousPeriod, x.Select(y => y.previousPeriod).ToHashSet()}});
}

# Aoc Step Configuration

In [ ]:
public static async Task<IEnumerable<AocConfiguration>> LoadAocStepConfigurationAsync(this IQuerySource querySource, int year, int month)
    => (await querySource.LoadParameterAsync<AocConfiguration>(year, month))
            .GroupBy(x => (x.AocType, x.Novelty), 
                    (k, v) => v.OrderByDescending(x => x.Year).ThenByDescending(x => x.Month).First()); 

In [ ]:
public static async Task<Dictionary<AocStep, AocConfiguration>> LoadAocStepConfigurationAsDictionaryAsync(this IQuerySource querySource, int year, int month) 
    => (await querySource.LoadAocStepConfigurationAsync(year, month))
            .ToDictionary(x => new AocStep(x.AocType, x.Novelty)); 